## 12. OpenAI の Chat API の Function calling 機能について

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### 12.1 Function calling の基本

In [2]:
import requests
import xml.etree.ElementTree as ET

res = requests.get("https://weather.tsukumijima.net/primary_area.xml")
root = ET.fromstring(res.text)
cities = root.findall(".//city")

city_dict = {}
for city in cities:
    city_dict[city.get("title")] = city.get("id")

def get_city_id(city):

    return city_dict[city]

In [3]:
import json

def get_current_weather(city):

    city_id = get_city_id(city)
    res = requests.get(f"https://weather.tsukumijima.net/api/forecast/city/{city_id}")
    root = json.loads(res.text)

    return json.dumps(root["forecasts"][0])

In [4]:
import openai

def run_conversation():

    # STEP 1

    messages = [
        {
            "role": "user", 
            "content": "現在の横浜の天気を教えてください。"
        }
    ]

    functions = [
        {
            "name": "get_current_weather",
            "description": "与えられた地域の現在の天気を取得する。",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "地域、例えば、東京、横浜",
                    }
                },
                "required": ["location"],
            }
        }
    ]

    openai_1st_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )

    print("=== STEP 1 の回答 ===")
    print(openai_1st_response)

    ai_message = openai_1st_response["choices"][0]["message"]

    # STEP 2

    if ai_message.get("function_call"):

        # STEP 3

        function_name = ai_message["function_call"]["name"]
        function_args = json.loads(ai_message["function_call"]["arguments"])
        function_response = globals()[function_name](function_args.get("location"))

        print("=== STEP 3 の回答 ===")
        print(function_response)

        # STEP 4

        messages.append(ai_message)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        openai_2nd_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
        )

        print("=== STEP 4 の回答 ===")
        print(openai_2nd_response)

        return openai_2nd_response["choices"][0]["message"]["content"]

result = run_conversation()
print(result)

=== STEP 1 の回答 ===
{
  "id": "chatcmpl-7on1OIb6JVcRW8hrnFh8rzOojorPw",
  "object": "chat.completion",
  "created": 1692339194,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"\u6a2a\u6d5c\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 100,
    "completion_tokens": 20,
    "total_tokens": 120
  }
}
=== STEP 3 の回答 ===
{"date": "2023-08-18", "dateLabel": "\u4eca\u65e5", "telop": "\u6674\u6642\u3005\u66c7", "detail": {"weather": "\u6674\u308c\u3000\u6642\u3005\u3000\u304f\u3082\u308a", "wind": "\u5357\u306e\u98a8\u3000\u6d77\u4e0a\u3000\u3067\u306f\u3000\u5357\u306e\u98a8\u3000\u3084\u3084\u5f37\u304f", "wave": "\uff11\u30e1\u30fc\u30c8\u30eb"}, "temperature": {"min": {"celsius": null, "fahrenheit": null}, "max": {"celsius

### 12.2 Function calling を使った LangChain の「OpenAI Functions Agent」

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType

tools = [
    Tool(
        name="get_current_weather",
        func=get_current_weather,
        description="与えられた地域（例えば、東京、横浜）の現在の天気を取得する。",
    )
]

llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0
)

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS)

result = agent.run("現在の横浜の天気を教えてください。")
print(result)

横浜の現在の天気は「晴時々曇り」です。南の風が海上でやや強く吹いており、波の高さは1メートルです。最高気温は35度です。降水確率は午前中と夕方に10%です。

![天気のアイコン](https://www.jma.go.jp/bosai/forecast/img/101.svg)


### 12.3 Function calling を応用した Extraction と Tagging

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}

user_message = """
アレックスは身長5フィートです。クラウディアはアレックスより1フィート背が高く、彼よりも高くジャンプします。クラウディアはブルネットで、アレックスはブロンドです。
アレックスの犬のフロスティはラブラドール種で、かくれんぼをするのが好きです。
"""

llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0
)
chain = create_extraction_chain(schema, llm)

result = chain.run(user_message)
print(json.dumps(result, indent=2, ensure_ascii=False))


[
  {
    "person_name": "アレックス",
    "person_height": 5,
    "person_hair_color": "ブロンド",
    "dog_name": "フロスティ",
    "dog_breed": "ラブラドール種"
  },
  {
    "person_name": "クラウディア",
    "person_height": 6,
    "person_hair_color": "ブルネット"
  }
]


### 12.4 1 度に複数の関数を実行できる LangChain の「OpenAI Multi Functions Agent」

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain

schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

user_message = "Estoy muy enojado con vos! Te voy a dar tu merecido!"

llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0
)
chain = create_tagging_chain(schema, llm)

result = chain.run(user_message)
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "sentiment": "enojado",
  "aggressiveness": 1,
  "language": "es"
}


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

tools = load_tools(["ddg-search"])
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = initialize_agent(tools, chat, agent=AgentType.OPENAI_MULTI_FUNCTIONS)

result = agent.run("What is the weather in Yokohama?")

print(result)

The current weather in Yokohama is 88 °F with scattered clouds. The forecast for today shows a maximum daytime temperature of 30 degrees Celsius and a minimum nighttime temperature of 27 degrees Celsius. There are heavy showers expected to change to clear by nighttime. In September, the weather in Yokohama is usually warm to hot and muggy, with an average temperature of 24.2 °C (75 °F). The coldest nights at the end of the month can drop to around 16 °C (61 °F).
